In [12]:
import ray
from ray.util.dask import ray_dask_get
import dask
import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd

ray.util.connect("127.0.0.1:10001")

dask.config.set(scheduler=ray_dask_get)

RuntimeError: Ray Client is already connected. Maybe you called ray.util.connect twice by accident?

In [2]:
path="s3://unext-datascience-prod/jobs/ippanreco/user_watched_sakuhins.csv.gz"
df = dd.read_csv(path)
# dd.read_csv(path).to_parquet("s3://unext-datascience/starfish/test/",engine="pyarrow")

/home/ray/anaconda3/lib/python3.7/site-packages/dask/dataframe/io/csv.py:474: UserWarning: Warning gzip compression does not support breaking apart files
Please ensure that each individual file can fit in memory and
use the keyword ``blocksize=None to remove this message``
Setting ``blocksize=None``
  "Setting ``blocksize=None``" % compression


In [3]:
df = df.repartition(npartitions=100).persist()

In [4]:
df["sakuhin_codes"] = df.sakuhin_codes.str.split('|')
df = df.explode("sakuhin_codes").persist()

In [10]:
df.head()

,user_multi_account_id,sakuhin_codes
0,C0000000047,SID0055552
1,C0000001165,SID0041951
1,C0000001165,SID0040184
1,C0000001165,SID0025866
1,C0000001165,SID0056125


In [11]:
df.to_parquet("s3://unext-datascience/starfish/test/",engine="pyarrow")

RaySystemError: System error: Ray has not been started yet. You can start Ray with 'ray.init()'.